In [1]:
import torch
from datafeed import RegressionDataset, ClassificationDataset
import torch.nn as nn
import torch.optim as optim
import math 
from model import FuncControlModel
import copy
# Define your model
import random 
# data is sinx , x is 0 to 1
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm import tqdm 
from torchmetrics import MeanAbsolutePercentageError
import datetime
from functions import FUNC_CLASSES
from func_manage import transfer_methods
from functions import *
import torchvision 
import torchvision.transforms as transforms
from func_prune import get_topk_funcs, remove_other_funcs
import inspect

In [2]:
# class별 함수이름 list로 반환
def get_class_functions(cls):
    class_functions = [func_name for func_name in dir(cls) if inspect.isfunction(getattr(cls, func_name))]
    return class_functions

# 함수별 입력 개수 반환
def get_num_input_args(cls, func_name):
    func = getattr(cls, func_name)
    num_args = len(inspect.signature(func).parameters)
    return num_args

# nan, inf 판별
def nan_or_inf(val):
    if torch.isnan(val).any() or torch.isinf(val).any():
        return 0
    else:
        return 1

### CASE1 & CASE2

In [3]:
def original_func_max(cls, type):
    instance = cls()
    result_list = []
    
    if type == 'case1':
        max_val = torch.tensor([[1e+50]])
    elif type == 'case2':
        max_val = torch.tensor([[1e-10]])

    random_num = torch.rand(1, 1) #입력 2개인 경우 하나는 랜덤 값
    class_functions = get_class_functions(cls) #class에 있는 func 이름 반환
    #print(f"class functions : {class_functions} || length: {len(class_functions)}\n")

    for func_name in class_functions:

        num_args = get_num_input_args(cls, func_name) - 1 #self 제외
        #print(f"<<<func_name: {func_name} | num_args: {num_args}>>>")
        
        if num_args == 1:
            temp = getattr(instance, func_name)(max_val)
            #print(f"num_args(1)-before | result: {temp}")
            if nan_or_inf(temp) == 1:
                result_list.append(1)
            else: 
                result_list.append(0)


        elif num_args == 2:

            temp = getattr(instance, func_name)(max_val, random_num)
            #print(f"num_args(2)-1-before | result: {temp}")

            temp2 = getattr(instance, func_name)(random_num, max_val)
            #print(f"num_args(2)-2-before | result: {temp2}")

            if nan_or_inf(temp) == 1 and nan_or_inf(temp2) == 1:
                result_list.append(1)
            else:
                result_list.append(0)
   

    return torch.tensor(result_list)

In [4]:
def original_func_min(cls, type):
    instance = cls()
    result_list = []

    if type == 'case1':
        min_val = torch.tensor([[-100000.0]])
    elif type == 'case2':
        min_val = torch.tensor([[-1e-10]])

    random_num = torch.rand(1, 1) #입력 2개인 경우 하나는 랜덤 값
    class_functions = get_class_functions(cls) #class에 있는 func 이름 반환
    #print(f"class functions : {class_functions} || length: {len(class_functions)}\n")

    for func_name in class_functions:

        num_args = get_num_input_args(cls, func_name) - 1 #self 제외
        #print(f"<<<func_name: {func_name} | num_args: {num_args}>>>")
        
        if num_args == 1:
            temp = getattr(instance, func_name)(min_val)
            #print(f"num_args(1)-before | result: {temp}")
            if nan_or_inf(temp) == True:
                result_list.append(1)
            else: 
                result_list.append(0)


        elif num_args == 2:

            temp = getattr(instance, func_name)(min_val, random_num)
            #print(f"num_args(2)-1-before | result: {temp}")

            temp2 = getattr(instance, func_name)(random_num, min_val)
            #print(f"num_args(2)-2-before | result: {temp2}")

            if nan_or_inf(temp) == True and nan_or_inf(temp2) == True:
                result_list.append(1)
            else:
                result_list.append(0)
   

    return torch.tensor(result_list)

### CASE3 & CASE4

In [5]:
def prime_func_max(cls, type):
    instance = cls()
    result_list = []

    if type == 'case3':
        max_val = torch.tensor([1e+50], requires_grad=True)
    elif type == 'case4':
        max_val = torch.tensor([1e-10], requires_grad=True)

    random_num = torch.rand(1, 1) #입력 2개인 경우 하나는 랜덤 값
    class_functions = get_class_functions(cls) #class에 있는 func 이름 반환


    
    for func_name in class_functions:
        num_args = get_num_input_args(cls, func_name) - 1 #self 제외
       
        if num_args == 1:

            y = getattr(instance, func_name)(max_val)
            y.backward()
            temp = max_val.grad

            if nan_or_inf(temp) == 1:
                result_list.append(1)
            else: 
                result_list.append(0)


        elif num_args == 2:
            y = getattr(instance, func_name)(max_val, random_num)
            y.backward()
            temp = max_val.grad
            #print(f"num_args(2)-1-before | result: {temp}")

            y2 = getattr(instance, func_name)(random_num, max_val)
            y2.backward()
            temp2 = max_val.grad
            #print(f"num_args(2)-2-before | result: {temp2}")

            if nan_or_inf(temp) == 1 and nan_or_inf(temp2) == 1:
                result_list.append(1)
            else:
                result_list.append(0)
   

    return torch.tensor(result_list)

In [6]:
def prime_func_min(cls, type):
    instance = cls()
    result_list = []

    if type == 'case3':
        min_val = torch.tensor([-100000.], requires_grad=True)
    elif type == 'case4':
        min_val = torch.tensor([-1e-10], requires_grad=True)

    random_num = torch.rand(1, 1) #입력 2개인 경우 하나는 랜덤 값
    class_functions = get_class_functions(cls) #class에 있는 func 이름 반환

    
    for func_name in class_functions:

        num_args = get_num_input_args(cls, func_name) - 1 #self 제외
        
        if num_args == 1:
            y = getattr(instance, func_name)(min_val)
            y.backward()
            temp = min_val.grad
            #print(f"num_args(1)-before | result: {temp}")
            if nan_or_inf(temp) == 1:
                result_list.append(1)
            else: 
                result_list.append(0)


        elif num_args == 2:

            y = getattr(instance, func_name)(min_val, random_num)
            y.backward()
            temp = min_val.grad
            #print(f"num_args(2)-1-before | result: {temp}")

            y2 = getattr(instance, func_name)(random_num, min_val)
            y2.backward()
            temp2 = min_val.grad
            #print(f"num_args(2)-2-before | result: {temp2}")

            if nan_or_inf(temp) == 1 and nan_or_inf(temp2) == 1:
                result_list.append(1)
            else:
                result_list.append(0)
   

    return torch.tensor(result_list)

In [7]:
class_list = [FuncPool, Trigonometric, InverseTrigonometric, Hyperbolic, InverseHyperbolic, Power, Fourier, Exponential, Mathmatical]
stable_dict = {}

for class_name in class_list:
    temp_list = []
    
    ## CASE 1
    case1_final_result = (original_func_max(class_name, 'case1') & original_func_min(class_name, 'case1'))

    ## CASE 2
    case2_final_result = (original_func_max(class_name, 'case2') & original_func_min(class_name, 'case2'))

    ## CASE 3
    case3_final_result = (prime_func_max(class_name, 'case3') & prime_func_min(class_name, 'case3'))

    ## CASE 4
    case4_final_result = (prime_func_max(class_name, 'case4') & prime_func_min(class_name, 'case4'))
 
    temp_list.append(case1_final_result)
    temp_list.append(case2_final_result)
    temp_list.append(case3_final_result)
    temp_list.append(case4_final_result)


    stable_dict[class_name.__name__] = temp_list

print(f"stable_dict: {stable_dict}")



stable_dict: {'FuncPool': [tensor([0, 0, 0, 1, 1, 0, 0]), tensor([1, 1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1, 1])], 'Trigonometric': [tensor([0, 0, 0, 0, 0]), tensor([1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 0]), tensor([1, 1, 1, 1, 1])], 'InverseTrigonometric': [tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1])], 'Hyperbolic': [tensor([1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1])], 'InverseHyperbolic': [tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1])], 'Power': [tensor([1, 0]), tensor([1, 1]), tensor([1, 1]), tensor([1, 1])], 'Fourier': [tensor([1, 1, 1]), tensor([1, 1, 1]), tensor([1, 1, 1]), tensor([1, 1, 1])], 'Exponential': [tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 1, 1, 1])], 'Mathmatical': [tensor([1, 1, 1, 1, 1, 1, 1, 1,